In [54]:
import cPickle as pickle
import pandas as pd
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

In [55]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
docid_data=pickle.load(open('document_results_allv_norm.pickle', 'rb'))

In [57]:
data=pickle.load(open("results_.pickle", 'rb'))

In [58]:
data.head(5)

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var,tfidf_fullvar,tfidf_HHI
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271,1.172770e-08,0.050491
1,0,137827,62,89,11872,0.014588,0.007497,0.012235,7.028444e-07,0.000000
2,0,137827,488,67,11872,0.016907,0.005644,0.014718,9.472815e-10,0.000000
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032,2.374237e-11,0.021899
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314,5.678782e-10,0.018403


In [59]:
methods=data.columns[5:]
method_list=methods.tolist()

In [60]:
method_list

['tfidf', 'tf', 'tfidf_var', 'tfidf_fullvar', 'tfidf_HHI']

In [61]:
docid_corpus_calc=defaultdict()
for docid in tqdm(docid_data.iterkeys()):
    docid_corpus_calc[docid]=defaultdict(list)
    for method in method_list:
        count=[0,0,0]
        count_weight=docid_data[docid][method]
        index=count_weight.index(max(count_weight))
        count[index]=1
        docid_corpus_calc[docid][method]=count

537it [00:00, 35799.75it/s]


In [62]:
docid_corpus_calc[137827]

defaultdict(list,
            {'tf': [1, 0, 0],
             'tfidf': [1, 0, 0],
             'tfidf_HHI': [1, 0, 0],
             'tfidf_fullvar': [1, 0, 0],
             'tfidf_var': [1, 0, 0]})

In [63]:
docid_corpus_fact=defaultdict()
df=pd.concat([data['docid'], data['corpus_id']],axis=1).drop_duplicates()
for _, row in df.iterrows():
    docid_corpus_fact[row['docid']]=[0,0,0]
    docid_corpus_fact[row['docid']][row['corpus_id']]=1

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
true_positive=defaultdict(list)

for method in method_list:
    true_positive[method]=[0,0,0]
    for docid in docid_corpus_fact.iterkeys():
        if cosine_similarity(docid_corpus_fact[docid], docid_corpus_calc[docid][method])==1:
            count=docid_corpus_fact[docid]
            index=count.index(max(count))
            true_positive[method][index]+=1
        
        

In [68]:
true_positive

defaultdict(list,
            {'tf': [258, 109, 136],
             'tfidf': [258, 107, 116],
             'tfidf_HHI': [241, 111, 145],
             'tfidf_fullvar': [259, 110, 128],
             'tfidf_var': [258, 107, 116]})

In [69]:
all_positive=[0,0,0]
for docid in docid_corpus_fact.iterkeys():
    for index in range(3):
        all_positive[index]+=docid_corpus_fact[docid][index]

In [70]:
all_selected=defaultdict(list)
for method in method_list:
    all_selected[method]=[0,0,0]
    for docid in docid_corpus_calc.iterkeys():
        for index in range(3):
            all_selected[method][index]+=docid_corpus_calc[docid][method][index]
    

In [71]:
all_selected['tf']

[275, 120, 142]

In [72]:
all_positive

[271, 114, 152]

In [73]:
methods_results=defaultdict()
for method in method_list:
    methods_results[method]=defaultdict(list)
    methods_results[method]['precision']=[0.0,0.0,0.0]
    methods_results[method]['recall']=[0.0,0.0,0.0]
    for index in range(3):
        methods_results[method]['precision'][index]+=float(true_positive[method][index])/all_selected[method][index]
        methods_results[method]['recall'][index]+=float(true_positive[method][index])/all_positive[index]
    
    

In [74]:
methods_results_macroavg=defaultdict()
for method in method_list:
    methods_results_macroavg[method]=defaultdict(float)
    methods_results_macroavg[method]['precision']=np.mean(methods_results[method]['precision'])
    methods_results_macroavg[method]['recall']=np.mean(methods_results[method]['recall'])
    methods_results_macroavg[method]['fscore']=2*(methods_results_macroavg[method]['precision']*
                                                    methods_results_macroavg[method]['recall'])/ (
                                                    methods_results_macroavg[method]['precision']+
                                                    methods_results_macroavg[method]['recall'])

In [75]:
methods_results_microavg=defaultdict(float)
for method in method_list:
    methods_results_microavg[method]=defaultdict(float)
    methods_results_microavg[method]['precision']=float(sum(true_positive[method]))/sum(all_selected[method])
    methods_results_microavg[method]['recall']=float(sum(true_positive[method]))/sum(all_positive)
    methods_results_microavg[method]['fscore']=2*(methods_results_microavg[method]['precision']*
                                                    methods_results_microavg[method]['recall'])/ (
                                                    methods_results_microavg[method]['precision']+
                                                    methods_results_microavg[method]['recall'])

In [76]:
methods_results_macroavg


defaultdict(None,
            {'tf': defaultdict(float,
                         {'fscore': 0.93452800271060832,
                          'precision': 0.93475387679613042,
                          'recall': 0.93430223775921972}),
             'tfidf': defaultdict(float,
                         {'fscore': 0.89028203353123803,
                          'precision': 0.89604303781133054,
                          'recall': 0.88459463542003836}),
             'tfidf_HHI': defaultdict(float,
                         {'fscore': 0.92637093938797221,
                          'precision': 0.91409904221600247,
                          'recall': 0.93897682397876603}),
             'tfidf_fullvar': defaultdict(float,
                         {'fscore': 0.92286914836838885,
                          'precision': 0.92483426307704741,
                          'recall': 0.92091236701840706}),
             'tfidf_var': defaultdict(float,
                         {'fscore': 0.89028203353123803,
   

In [77]:
methods_results_macroavg

defaultdict(None,
            {'tf': defaultdict(float,
                         {'fscore': 0.93452800271060832,
                          'precision': 0.93475387679613042,
                          'recall': 0.93430223775921972}),
             'tfidf': defaultdict(float,
                         {'fscore': 0.89028203353123803,
                          'precision': 0.89604303781133054,
                          'recall': 0.88459463542003836}),
             'tfidf_HHI': defaultdict(float,
                         {'fscore': 0.92637093938797221,
                          'precision': 0.91409904221600247,
                          'recall': 0.93897682397876603}),
             'tfidf_fullvar': defaultdict(float,
                         {'fscore': 0.92286914836838885,
                          'precision': 0.92483426307704741,
                          'recall': 0.92091236701840706}),
             'tfidf_var': defaultdict(float,
                         {'fscore': 0.89028203353123803,
   

In [81]:
pickle.dump(methods_results_macroavg, open('methods_results_macroavg_norm.pickle', 'wb'))
pickle.dump(methods_results_microavg, open('methods_results_microavg_norm.pickle', 'wb'))

In [80]:
methods_results_microavg

defaultdict(float,
            {'tf': defaultdict(float,
                         {'fscore': 0.9366852886405957,
                          'precision': 0.9366852886405959,
                          'recall': 0.9366852886405959}),
             'tfidf': defaultdict(float,
                         {'fscore': 0.8957169459962756,
                          'precision': 0.8957169459962756,
                          'recall': 0.8957169459962756}),
             'tfidf_HHI': defaultdict(float,
                         {'fscore': 0.925512104283054,
                          'precision': 0.925512104283054,
                          'recall': 0.925512104283054}),
             'tfidf_fullvar': defaultdict(float,
                         {'fscore': 0.925512104283054,
                          'precision': 0.925512104283054,
                          'recall': 0.925512104283054}),
             'tfidf_var': defaultdict(float,
                         {'fscore': 0.8957169459962756,
                     